# Part 1: Create database set
By inputting a target gene, a nested dictionary will be created, with each key being the name of a cancer and each value being a dictionary containing the genomic, proteomic, transcriptomic, and phosphoproteomic data. In other words, the dictionary will be:

{
    cancer : {   
    
            "gen" : <cancer's genomic data>,
            "prot" : <cancer's proteomic data>,
            "trans": <cancer's transcriptomic data>,
            "phos": <cancer's phosphoproteomic data>
        }
    ... for each cancer
}

In [1]:
import cptac
import cptac.pancan as pc
import numpy as np
import cptac.utils as ut
import plot_utils as plt
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset_names = pc.list_datasets(False)
dataset_names  = dataset_names.split('\n')
dataset_names

['PancanBrca',
 'PancanCcrcc',
 'PancanCoad',
 'PancanGbm',
 'PancanHnscc',
 'PancanLscc',
 'PancanLuad',
 'PancanOv',
 'PancanUcec',
 'PancanPdac']

In [3]:
target_gene = "TP53"

In [4]:
datasets = {}
for name in dataset_names:
    exec(f"cancer = pc.{name}(no_internet = True)")  
    genotype_data = cancer.get_genotype_all_vars(mutations_genes = target_gene, omics_source = "washu")
    
    proteomic_data = cancer.get_proteomics(source = 'umich')
    proteomic_data = proteomic_data.droplevel("Database_ID", axis = 1)
    
    phospho_data = cancer.get_phosphoproteomics(source = 'umich')
    phospho_data.loc[:, phospho_data.columns.get_level_values(0) == target_gene].droplevel(["Database_ID", "Peptide"], axis = 1)
    
    transcript_data = cancer.get_transcriptomics(source = "washu")
    transcript_data = transcript_data.droplevel("Database_ID", axis = 1)
    
    datasets[name[6:]] = {
                        "gen" : genotype_data,
                        "prot": proteomic_data,
                        "trans": transcript_data,
                        "phos" : phospho_data
                         }    

/home/sdsquires/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
/home/sdsquires/anaconda3/lib/python3.9/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 68 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)
cptac warning: washu_CNV data was not found for the following samples, so washu_CNV data columns were filled with NaN for these samples: 01BR044, 604 (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 104 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 44 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 67 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


...

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 9 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 50 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 5 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 73 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 41 samples for the TP53 gene (/home/sdsquires/anaconda3/lib/python3.9/site-packages/cptac/pancan/pancandataset.py, line 174)


# Part 2: Select mutations
Determine which cancers have mutation types with rates above 20%.

To facilitate that, the mutation types from the get_genotype_all_vars must be grouped into functional types.


In [16]:
mutation_types = ["Deletion", "Amplification", "Truncation", "Indel", "Missense"]

In [6]:
# Grouping the mutation types for each cancer
for name, dataset in datasets.items():
    genotypes = dataset["gen"]
    genotypes.replace(["Nonsense_Mutation", "Frame_Shift_Ins", "Frame_Shift_Del"], "Truncation", inplace = True)
    genotypes.replace(["In_Frame_Del", "In_Frame_Ins"], "Indel", inplace = True)
    genotypes.replace(["Missense_Mutation", "Wildtype_Tumor"], ["Missense", "Wildtype"], inplace = True)
    genotypes = genotypes.loc[genotypes["Mutation"] != "Silent"]
    datasets[name]["gen"] = genotypes


In [7]:
# Create a bargraph displaying the mutation types

In [17]:
# Determine which cancers have mutation rates over 20%
variants_to_analyze = {}
for name, dataset in datasets.items():
    mut_type_freq = dataset["gen"]["Mutation"].value_counts() / len(dataset["prot"])
    mut_type_freq = mut_type_freq.loc[mut_type_freq >= 0.2]
    variants_to_analyze[name] = set(mut_type_freq.index.tolist())
    variants_to_analyze[name].discard("Wildtype")
print(str(variants_to_analyze))

{'Brca': {'Deletion'}, 'Ccrcc': set(), 'Coad': {'Deletion'}, 'Gbm': set(), 'Hnscc': {'Missense'}, 'Lscc': set(), 'Luad': set(), 'Ov': {'Missense', 'Deletion'}, 'Ucec': set(), 'Pdac': set()}


# Part 3: Cis effects
Determines which genes that, given a mutation, will change the expression in their own proteomics, transcriptomics, or phosphoproteomics.This analysis is repeated for every cancer, and every mutation type.

Writes out a file containing the infomation for each mutation and cancer, and creates boxplot summarizing the data

In [64]:
from scipy.stats import ttest_ind
omics_key = {"prot":"proteomics", "trans":"transcriptomics","phos":"phosphoproteomics"}

In [68]:
# Create a function that will analyze the protein cis effects for a given mutation type
def analyze_cis_mutation(cancer_type, omics_type, write_out = True, pval_only = True):
    """Analyzes the cis effects for a given mutation type across all cancers.
    
    Parameters:
    mutation_type (str): The mutation type to analyze, as a string. Possible values given in mutation_types.
    omics_type (str): The omics to analyze. Possible values are in list ["prot", "trans", "phos"]
    write_out (bool, optional): Whether to write out the data to a separate file.
    pval    result = {'Mutation':[], 'P_value':[]}_only (bool, optional): If write_out, only writes out the p_value for each analysis. If not write_out, has no effect.
    
    Returns the output of scipy.ttest_ind() between "Wildtype" and mutation_type tumors. If analyzing phosphoproteomics, returns a list of such outputs
    """
    if write_out:
        # Clears the target file
        file_name = cancer_type.capitalize() + '_' + omics_key[omics_type].capitalize() + 's.txt'
        with open("Cis_Effect_output/" + file_name, 'w') as out_file: pass
        
    for mutation_type in mutation_types:
        if mutation_type not in variants_to_analyze[cancer_type]: continue
        print(mutation_type)
            
        if omics_type != "phos":
            genotype_and_omic = pd.merge(dataset["gen"], dataset[omics_type], left_index = True, right_index = True)
        
        genotype_and_omic = genotype_and_omic.loc[genotype_and_omic["Mutation"].isin(["Wildtype", mutation_type])]
        
        print(f"{omics_type} {mutation_type}s for {name}:")
        print(ut.wrap_ttest(df=genotype_and_omic, label_column="Mutation", cancer_type.capitalize() + '_' + omics_key[omics_type].capitalize()
                            comparison_columns=[target_gene], correction_method = "fdr_bh", return_all=True))
        print()
        

            
            
            
            
            
            
            
            
            
        
#         # Separate the wildtype and mutated samples
#         genotype_and_omic = pd.merge(dataset["gen"], dataset[omics_type], left_index = True, right_index = True)
#         mutation = genotype_and_omic.loc[genotype_and_omic["Mutation"] == mutation_type][target_gene].dropna()
#         wildtype = genotype_and_omic.loc[genotype_and_omic["Mutation"] == "Wildtype"][target_gene].dropna()
#         print(len(mutation))
#         print(len(wildtype))
        
#         # Phosphoproteomics must be analyzed at each site in the gene
#         if omics_type != "phos":
#             result = ttest_ind(mutation, wildtype)[1]
#         else:
#             result = [(site, ttest_ind(mutation[site], wildtype[site])[1]) for site in mutation.columns if site in wildtype.columns]
                
                
        ### Write out the data (not part of computation)###
#         if write_out:
#             output_string = ""
#             if not pval_only:
#                 output_string += f"{name}\n"
#                 output_string += f"Number of {mutation_type}s = {len(mutation)}\n"
#                 output_string += f"Number of Wildtypes = {len(wildtype)}\n"
#             if omics_type != "phos":
#                 output_string += f"{name} {mutation_type} P-value = {result}\n\n"
#             else:
#                 output_string += f"{name} {mutation_type} P-values:\n"
#                 for site in result:
#                     output_string += f"   {site[0]}: {site[1]}\n"
#                 ouput_string += '\n'
#             with open("Cis_Effect_output/" + file_name, "a") as out_file:
#                 print(output_string)
#                 out_file.write(output_string) 
                
#     try:
#         return result
#     except UnboundLocalError:
#         return



In [91]:
# Create a function that will analyze the protein cis effects for a given mutation type
def analyze_cis_mutation(omics_type, write_out = True, pval_only = True):
    """Analyzes the cis effects for a given mutation type across all cancers.
    
    Parameters:
    omics_type (str): The omics to analyze. Possible values are in list ["prot", "trans", "phos"]
    write_out (bool, optional): Whether to write out the data to a separate file.
    pval_only (bool, optional): If write_out, only writes out the p_value for each analysis. If not write_out, has no effect.
    
    Returns the output of scipy.ttest_ind() between "Wildtype" and mutation_type tumors. If analyzing phosphoproteomics, returns a list of such outputs
    """
    if write_out:
        # Clears the target file
        file_name =  omics_key[omics_type].capitalize() + '_Mutations.txt'
        with open("Cis_Effect_output/" + file_name, 'w') as out_file: pass
        
    result = {'Cancer':[], 'Mutation':[], 'P_Value':[]}
    if omics_type == "phos":
        result['Site']=[]
    for cancer_type, dataset in datasets.items():
        for mutation_type in mutation_types:
            if mutation_type not in variants_to_analyze[cancer_type]: continue
            print(mutation_type)

            if omics_type != "phos":
                genotype_and_omic = pd.merge(dataset["gen"], dataset[omics_type], left_index = True, right_index = True)
                genotype_and_omic = genotype_and_omic.loc[genotype_and_omic["Mutation"].isin(["Wildtype", mutation_type])]

                result['Cancer'].append(cancer_type)
                result['Mutation'].append(mutation_type)
                result['P_Value'].append(ut.wrap_ttest(df=genotype_and_omic, label_column="Mutation", 
                                                   comparison_columns=[target_gene], correction_method = "fdr_bh", 
                                                   return_all=True)["P_Value"][0])
    return pd.DataFrame(result)

In [92]:
analyze_cis_mutation("prot",True, False)

Deletion
Deletion
Missense
Deletion
Missense


,Cancer,Mutation,P_Value
0,Brca,Deletion,0.262125
1,Coad,Deletion,0.011910
2,Hnscc,Missense,0.000002
3,Ov,Deletion,0.360784
4,Ov,Missense,0.001135


In [58]:
#Now, run the analysis for every mutation_type, for every omics_type
for omics_type in ["prot", "trans", "phos"]:
    for mutation_type in mutation_types:
        analyze_cis_mutation(omics_type, mutation_type, write_out = True, pval_only = True)

In [ ]:
# Create a boxplot for each type, either in the function or outside of it
# Corrections must be performed to reduce the FDR

# Part 4: Trans Effects
Determines which genes that, given a mutation, will change the expression in another protein's proteomics, transcriptomics, or phosphoproteomics.This analysis is repeated for every cancer, and every mutation type.

To reduce the computational intensity and the corrections needed, the scope will be restricted to all genes whose corresponding proteins have a known interaction with the proteins corresponding to the target_gene. *(For example, Given the target_gene 'TP53', the set of genes will be all those whose proteins share a pathway with TP53.)*

Writes out a file containing the infomation for each mutation and cancer, and creates boxplot summarizing the data

In [ ]:
paths = ut.get_pathways_with_proteins(proteins = target_gene, database = 'wikipathways')
paths

In [ ]:
interacting_proteins = ut.get_proteins_in_pathways(paths["pathway_id"], database = 'wikipathways')
interacting_proteins = list(interacting_proteins["member"])

genes_measured = list(datasets["Brca"]["prot"].columns)
new_genes = sorted(list(set([protein for protein in interacting_proteins if protein in genes_measured])))

In [ ]:
interacting_proteins = set(interacting_proteins)
print(len(interacting_proteins))
genes_measured = set(genes_measured)
if target_gene in new_genes: new_genes.remove(target_gene)
print(len(genes_measured))
print(len(new_genes))
new_genes

In [ ]:
# Create a function that will analyze the protein cis effects for a given mutation type
def analyze_trans_mutation(omics_type, mutation_type, gene_list, write_out = True, pval_only = True):
    """Analyzes the trans effects for a given mutation type across all cancers.
    
    Parameters:
    mutation_type (str): The mutation type to analyze, as a string. Possible values given in mutation_types.
    omics_type (str): The omics to analyze. Possible values are in list ["prot", "trans", "phos"]
    gene_list (list[str]): A list of genes with proteins in the same pathway as those from target_gene
    write_out (bool, optional): Whether to write out the data to a separate file.
    pval_only (bool, optional): If write_out, only writes out the p_value for each analysis. If not write_out, has no effect.
    
    Returns a list of the outputs of scipy.ttest_ind() between "Wildtype" and mutation_type tumors, for each of the genes. If analyzing phosphoproteomics, returns a list of lists, in the form [[(site_name, pval) for each site] for each entry]
    """
    if write_out:
        # Clears the target file
        file_name = omics_key[omics_type].capitalize() + '_' + mutation_type.capitalize() + 's.txt'
        with open("Trans_Effect_output/" + file_name, 'w') as out_file: pass
    for name, dataset in datasets.items():
        if mutation_type not in variants_to_analyze[name]: continue
        
        # Separate the wildtype and mutated samples
        genotype_and_omic = pd.merge(dataset["gen"], dataset[omics_type], left_index = True, right_index = True)
        mutation = genotype_and_omic.loc[genotype_and_omic["Mutation"] == mutation_type].dropna()
        wildtype = genotype_and_omic.loc[genotype_and_omic["Mutation"] == "Wildtype"].dropna()
        
        # Phosphoproteomics must be analyzed at each site in the gene
        if omics_type != "phos":
            result = [ttest_ind(mutation[trans_gene].dropna(), wildtype[trans_gene].dropna())[1] for trans_gene in gene_list]
        else:
            result = [(site, ttest_ind(mutation[trans_gene][site].dropna(), wildtype[trans_gene][site])[1]) for site in trans_gene for trans_gene in mutation.columns if site in wildtype[trans_gene].columns]
                
        if write_out:
            output_string = ""
            if not pval_only:
                output_string += f"{name}\n"
                output_string += f"Number of {mutation_type}s = {len(mutation)}\n"
                output_string += f"Number of Wildtypes = {len(wildtype)}\n"
            if omics_type != "phos":
                output_string += f"{name} {mutation_type} P-value = {result}\n\n"
            else:
                output_string += f"{name} {mutation_type} P-values:\n"
                for site in result:
                    output_string += f"   {site[0]}: {site[1]}\n"
                ouput_string += '\n'
            with open("Trans_Effect_output/" + file_name, "a") as out_file:
                print(output_string)
                out_file.write("output_string") 
                
    try:
        return result
    except UnboundLocalError:
        return

    

In [ ]:
#Now, run the analysis for every mutation_type, for every omics_type
for omics_type in ["prot", "trans", "phos"]:
    for mutation_type in mutation_types:
        analyze_trans_mutation(omics_type, mutation_type, new_genes, write_out = True, pval_only = False)

In [ ]:
with open("deletion_trans_data.txt", 'w') as outFile:
    all_significant = {}
    for name, dataset in datasets.items():
        if name not in deletion_sets: continue
        print(name + '\n')
        outFile.write(name + '\n')
        significant_changes = {}
        for trans_gene in interacting_proteins:
            genotype_and_prot = dataset[0].merge(dataset[1], left_index = True, right_index = True)
            if trans_gene not in genotype_and_prot: continue
            deletions = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Deletion"][trans_gene].dropna()
            wildtypes = genotype_and_prot.loc[genotype_and_prot["Mutation"] == "Wildtype_Tumor"][trans_gene].dropna()
            result = ttest_ind(deletions, wildtypes)
            print(trans_gene + " P-value with " + name + " deletion: " + str(result[1]))
            outFile.write(name + " deletion P-value: " + str(result[1]))
        print('\n')
        outFile.write('\n')
